In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import requests
from io import BytesIO
import time
from tqdm import tqdm
import os
import logging
import gc
from itertools import product
import json
from datetime import datetime
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import multiprocessing
from google.colab import drive
drive.mount('/content/gdrive')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

def save_best_model(model, label_to_idx, config, val_metrics, epoch, best_val_loss, best_model_filename):
    if val_metrics['loss'] < best_val_loss:
        best_val_loss = val_metrics['loss']
        config_filename = f"model_lr_{config['hyperparameters']['learning_rate']}_bs_{config['hyperparameters']['batch_size']}_es_{config['hyperparameters']['image_size']}.pth"
        model_path = os.path.join(config['paths']['absolute_path'], config_filename)

        torch.save(model.state_dict(), model_path)

        label_to_idx_filename = model_path.replace('.pth', '_label_to_idx.json')
        with open(label_to_idx_filename, 'w') as f:
            json.dump(label_to_idx, f)

        logger.info(f"Best model saved at epoch {epoch} with loss {val_metrics['loss']}")

    return best_val_loss

class Config:
    @staticmethod
    def validate_config(config):
        """
        Validates the provided configuration to ensure all necessary paths, files, and parameters are correctly defined.
        - Checks for required paths and files in the dataset.
        - Ensures that hyperparameters like batch size, learning rate, and image size are positive.
        """

        required_paths = ['absolute_path', 'dataset_path']
        required_files = ['train_file', 'validation_file']
        required_columns = ['feature_col', 'label_col']

        # Validate paths
        for path in required_paths:
            if not os.path.exists(config['paths'][path]):
                raise ValueError(f"Path not found: {config['paths'][path]}")

        # Validate files
        for file in required_files:
            file_path = os.path.join(config['paths']['dataset_path'], config['filenames'][file])
            if not os.path.exists(file_path):
                raise ValueError(f"File not found: {file_path}")

        # Validate hyperparameters
        if config['hyperparameters']['batch_size'] <= 0:
            raise ValueError("Batch size must be positive")
        if config['hyperparameters']['learning_rate'] <= 0:
            raise ValueError("Learning rate must be positive")
        if config['hyperparameters']['image_size'] <= 0:
            raise ValueError("Image size must be positive")

        return True

class CustomImageDataset(Dataset):
    """
    A custom PyTorch Dataset for loading images from URLs, with optional caching of images locally.
    """

    # Initializes the dataset, reads the CSV file, and prepares label mappings.
    def __init__(self, csv_file, config, transform=None, cache_dir=None):
        self.data = pd.read_csv(os.path.join(config['paths']['dataset_path'], csv_file))
        self.transform = transform
        self.feature_col = config['columns']['feature_col']
        self.label_col = config['columns']['label_col']
        # Creates a sorted list of unique classes (labels) from the dataset.
        self.classes = sorted(self.data[self.label_col].unique())
        # Maps each class label to an index for numerical representation.
        self.label_to_idx = {label: idx for idx, label in enumerate(self.classes)}
        # Sets the directory for caching images locally.
        self.cache_dir = cache_dir

        if cache_dir and not os.path.exists(cache_dir):
            os.makedirs(cache_dir)

    # Returns the number of samples in the dataset.
    def __len__(self):
        return len(self.data)

    # Retrieves the image from the cache or downloads it from a URL.
    def _load_image_from_cache(self, url, idx):
        if self.cache_dir:
            cache_path = os.path.join(self.cache_dir, f"img_{idx}.jpg")
            if os.path.exists(cache_path):
                return Image.open(cache_path).convert('RGB')

        response = requests.get(url, timeout=10)
        if response.status_code != 200:
            raise ValueError(f"Failed to fetch image: HTTP {response.status_code}")

        img = Image.open(BytesIO(response.content)).convert('RGB')

        if self.cache_dir:
            img.save(cache_path)

        return img

    # Loads an image and its corresponding label
    def __getitem__(self, idx):
        try:
            img_url = self.data.iloc[idx][self.feature_col]
            label = self.data.iloc[idx][self.label_col]

            img = self._load_image_from_cache(img_url, idx)

            if self.transform:
                img = self.transform(img)

            label_idx = self.label_to_idx[label]
            return img, label_idx

        except Exception as e:
            logger.error(f"Error loading image at index {idx}: {str(e)}")
            raise

def clear_gpu_memory():
    """
    Clears GPU memory to avoid memory overflow issues during training.
    - Uses PyTorch's built-in functions to release GPU memory.
    """
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()

class ModelTrainer:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.metrics_history = []
        self.label_to_idx = None

    # Prepares data augmentations and preprocessing steps for training and validation datasets.
    def _create_transforms(self):
        # Define transformations for the training dataset.
        # For our small dataset, more augmentations could help avoiding overfitting.
        # Each time an image is passed through the data loader,
        # these transformations are applied with randomized parameters.
        train_transform = transforms.Compose([
            # Resizes the image to a square defined by the configured image size.
            transforms.Resize((self.config['hyperparameters']['image_size'],
                           self.config['hyperparameters']['image_size'])),
            # Randomly flips the image horizontally.
            transforms.RandomHorizontalFlip(),
            # Randomly rotates the image by up to 15 degrees.
            transforms.RandomRotation(15),
            # Randomly changes the brightness, contrast, and saturation of the image.
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(), # Converts the image into a PyTorch tensor.
            # Normalizes the image using the specified mean
            # and standard deviation values (pre-trained model standards).
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])

        # Define transformations for the validation dataset (no data augmentation).
        val_transform = transforms.Compose([
            transforms.Resize((self.config['hyperparameters']['image_size'],
                           self.config['hyperparameters']['image_size'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
        ])

        return train_transform, val_transform

    # Creates PyTorch DataLoader objects for efficient data loading during training and validation.
    def _create_dataloaders(self, train_transform, val_transform):
        train_dataset = CustomImageDataset(
            self.config['filenames']['train_file'],
            self.config,
            transform=train_transform
        )

        self.label_to_idx = train_dataset.label_to_idx  # Store the mapping

        validation_dataset = CustomImageDataset(
            self.config['filenames']['validation_file'],
            self.config,
            transform=val_transform
        )

        num_workers = min(multiprocessing.cpu_count(), 4)

        train_loader = DataLoader(
            train_dataset,
            batch_size=self.config['hyperparameters']['batch_size'],
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True
        )

        validation_loader = DataLoader(
            validation_dataset,
            batch_size=self.config['hyperparameters']['batch_size'],
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )

        return train_loader, validation_loader, train_dataset.classes

    # Configures a ResNet50 model with fine-tuning of specific layers for the given number of classes.
    def _create_model(self, num_classes):
        # Loads a pre-trained ResNet-50 model with weights trained on the ImageNet dataset.
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        # Replaces the fully connected (fc) layer with a new sequential layer consisting of:
        # 1. Dropout layer (with a probability of 0.5) to reduce overfitting.
        # 2. Linear layer to adjust the output to match the number of classes in the dataset.
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(model.fc.in_features, num_classes)
        )
        # Moves the model to the specified device (GPU) for training.
        model = model.to(self.device)

        # Freezes the early layers of the model to prevent
        # their weights from being updated during training.
        for param in model.parameters():
            param.requires_grad = False

        # Unfreezes the parameters of the final convolutional block (layer4)
        # to allow fine-tuning.
        for param in model.layer4.parameters():
            param.requires_grad = True
        for param in model.fc.parameters():
            param.requires_grad = True

        return model

    # Performs one epoch of training and computes training metrics like loss and accuracy.
    def _train_epoch(self, model, train_loader, criterion, optimizer):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        all_preds = [] # List to store all predicted labels for the epoch.
        all_labels = [] # List to store all true labels for the epoch.

        # Loops through the training data loader batch by batch.
        for inputs, labels in tqdm(train_loader, desc='Training'):
            # Moves the inputs and labels to the configured device (GPU).
            inputs, labels = inputs.to(self.device), labels.to(self.device)

            # Clears the gradients of the optimizer to prepare for the current batch.
            optimizer.zero_grad()
            # Passes the inputs through the model to obtain outputs (predictions).
            outputs = model(inputs)
            # Computes the loss between the predictions and the true labels.
            loss = criterion(outputs, labels)
            # Backpropagates the loss to compute gradients for all trainable parameters.
            loss.backward()

            # Clips gradients to avoid exploding gradients during backpropagation.
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step() # Updates the model parameters using the optimizer.

            # Accumulates the loss for reporting the average loss over the epoch.
            running_loss += loss.item()
            # Gets the predicted class labels for the batch by taking the index
            # of the maximum value in each output vector.
            _, predicted = torch.max(outputs.data, 1)
            # Updates the total number of labels and the count of correct
            # predictions for accuracy calculation.
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Appends the predicted and true labels for this batch
            # to the lists for further analysis.
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        return {
            'loss': running_loss / len(train_loader),
            'accuracy': 100 * correct / total,
            'predictions': np.array(all_preds),
            'labels': np.array(all_labels)
        }

    # Evaluates the model on the validation dataset and computes metrics like precision, recall, and F1-score.
    def _validate(self, model, validation_loader, criterion):
        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in validation_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        return {
            'loss': running_loss / len(validation_loader),
            'accuracy': 100 * correct / total,
            'predictions': np.array(all_preds),
            'labels': np.array(all_labels)
        }

    def train(self, patience=4):
        try:
            train_transform, val_transform = self._create_transforms()
            train_loader, validation_loader, classes = self._create_dataloaders(
                train_transform, val_transform)

            model = self._create_model(len(classes))
            criterion = nn.CrossEntropyLoss()
            optimizer = torch.optim.Adam([
                {'params': model.fc.parameters(),
                'lr': self.config['hyperparameters']['learning_rate'] * 10},
                {'params': model.layer4.parameters(),
                'lr': self.config['hyperparameters']['learning_rate']}
            ])
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', patience=1, factor=0.1)

            best_val_loss = float('inf')
            early_stop_counter = 0
            epoch_metrics = []

            for epoch in range(self.config['hyperparameters']['num_epochs']):
                epoch_start = time.time()

                train_metrics = self._train_epoch(model, train_loader, criterion, optimizer)
                clear_gpu_memory()
                val_metrics = self._validate(model, validation_loader, criterion)

                precision, recall, f1, _ = precision_recall_fscore_support(
                    val_metrics['labels'],
                    val_metrics['predictions'],
                    average='weighted',
                    zero_division=0
                )

                val_metrics.update({
                    'precision': precision,
                    'recall': recall,
                    'f1': f1
                })

                epoch_time = time.time() - epoch_start

                epoch_metrics.append({
                    'epoch': epoch + 1,
                    'train': train_metrics,
                    'validation': val_metrics,
                    'time': epoch_time
                })

                scheduler.step(val_metrics['accuracy'])

                best_val_loss = save_best_model(
                    model, self.label_to_idx, self.config, val_metrics,
                    epoch + 1, best_val_loss, "best_model.pth"
                )

                if val_metrics['loss'] < best_val_loss:
                    best_val_loss = val_metrics['loss']
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1

                if early_stop_counter >= patience:
                    logger.info("Early stopping triggered.")
                    break

            return epoch_metrics

        except Exception as e:
            logger.error(f"Training failed: {str(e)}")
            raise


    # Saves training results and configurations for reproducibility and analysis.
    def track_training_results(self, config, metrics):
        final_epoch_metrics = metrics[-1]
        return {
            'timestamp': datetime.now().isoformat(),
            'configuration': {
                'learning_rate': config['hyperparameters']['learning_rate'],
                'batch_size': config['hyperparameters']['batch_size'],
                'image_size': config['hyperparameters']['image_size'],
                'num_epochs': config['hyperparameters']['num_epochs']
            },
            'performance': {
                'final_accuracy': final_epoch_metrics['validation']['accuracy'],
                'final_loss': final_epoch_metrics['validation']['loss'],
                'precision': final_epoch_metrics['validation']['precision'],
                'recall': final_epoch_metrics['validation']['recall'],
                'f1_score': final_epoch_metrics['validation']['f1'],
                'training_time': final_epoch_metrics['time']
            }
        }

def load_previous_results(results_file):
    """
    Reads and parses the JSON file storing previous training results to avoid redundant training.
    - Returns a list of past results and tested parameter configurations.
    """
    try:
        with open(results_file, 'r') as f:
            all_results = json.load(f)
            # Extract the configurations that have been already tested
            tried_params = [result['configuration'] for result in all_results]
            return all_results, tried_params
    except (FileNotFoundError, json.JSONDecodeError):
        # If the file doesn't exist or is empty, return empty list
        return [], []

def grid_search(param_grid, base_config):
    """
    Performs hyperparameter tuning using a grid search over the parameter space.
    - Loads previously tested configurations to skip redundant experiments.
    - For each parameter combination:
        - Updates the configuration.
        - Trains the model and tracks metrics.
        - Saves results in a JSON file for reproducibility.
    - Returns all results from the search.
    """
    results_file = base_config['paths']['absolute_path'] + 'training_results.json'

    # Load previous results to avoid re-testing the same combinations
    all_results, tried_params = load_previous_results(results_file)

    # Generate all combinations of hyperparameters
    param_combinations = [dict(zip(param_grid.keys(), v)) for v in product(*param_grid.values())]

    # Filter out already tried combinations
    remaining_combinations = [params for params in param_combinations if params not in tried_params]

    print(remaining_combinations)

    # If no combinations are left to test
    if not remaining_combinations:
        print("All parameter combinations have already been tested.")
        return all_results

    # Proceed with the remaining combinations
    for params in remaining_combinations:
        current_config = base_config.copy()
        current_config['hyperparameters'].update(params)

        trainer = ModelTrainer(current_config)
        metrics = trainer.train()

        results = trainer.track_training_results(current_config, metrics)

        print(f"Configuration: {params}")
        print(f"Training Results: {results}")

        # Append the result to the list
        all_results.append(results)

        # Write the updated results list back to the file
        with open(results_file, 'w') as f:
            json.dump(all_results, f, indent=2)

    return all_results

def main():
    """
    The main entry point for running the training pipeline.
    - Sets up the base configuration and parameter grid.
    - Creates necessary directories.
    - Executes a grid search for hyperparameter tuning.
    - Logs results upon completion.
    """
    # Base configuration
    base_config = {
        'paths': {
            'absolute_path': "/content/gdrive/My Drive/Projects/ResNet/",
            'dataset_path': "/content/gdrive/My Drive/Projects/ResNet/Datasets/"
        },
        'filenames': {
            'train_file': 'train_set_400.csv',
            'validation_file': 'validation_set_400.csv'
        },
        'columns': {
            'feature_col': 'Image',
            'label_col': 'Category'
        },
        'hyperparameters': {  # this is just a template
            'learning_rate': 1e-4,
            'num_epochs': 10,
            'batch_size': 32,
            'image_size': 224
        }
    }

    # Parameter grid for search
    param_grid = {
        'learning_rate': [1e-3, 5e-4, 1e-4],
        'batch_size': [1, 16, 32, 64, 128, 256],
        'image_size': [100, 200, 400],
        'num_epochs': [3, 6, 12, 24, 30]
    }

    try:
        # Create necessary directories
        os.makedirs(base_config['paths']['absolute_path'], exist_ok=True)
        os.makedirs(base_config['paths']['dataset_path'], exist_ok=True)

        # Run grid search
        results = grid_search(param_grid, base_config)

        logger.info("\nGrid Search Completed! Check the training_results.json")

    except Exception as e:
        logger.error(f"Error in main: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[{'learning_rate': 0.0001, 'batch_size': 32, 'image_size': 200, 'num_epochs': 3}]


Training: 100%|██████████| 27/27 [02:31<00:00,  5.62s/it]


Configuration: {'learning_rate': 0.0001, 'batch_size': 32, 'image_size': 200, 'num_epochs': 3}
Training Results: {'timestamp': '2024-12-28T05:36:56.900840', 'configuration': {'learning_rate': 0.0001, 'batch_size': 32, 'image_size': 200, 'num_epochs': 3}, 'performance': {'final_accuracy': 44.44444444444444, 'final_loss': 1.8288670778274536, 'precision': 0.4747777777777778, 'recall': 0.4444444444444444, 'f1_score': 0.43662403605951994, 'training_time': 194.923321723938}}


# Predictions Phase

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import requests
from io import BytesIO
import json
import logging
import os
import time
from google.colab import drive
drive.mount('/content/gdrive')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('prediction.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class TestImageDataset(Dataset):
    """Custom Dataset for loading test images"""
    def __init__(self, csv_file, feature_col, transform=None):
        self.data = pd.read_csv(csv_file)
        self.feature_col = feature_col
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            img_url = self.data.iloc[idx][self.feature_col]

            # Download and open image
            response = requests.get(img_url, timeout=10)
            if response.status_code != 200:
                raise ValueError(f"Failed to fetch image: HTTP {response.status_code}")

            img = Image.open(BytesIO(response.content)).convert('RGB')

            if self.transform:
                img = self.transform(img)

            return img, idx

        except Exception as e:
            logger.error(f"Error loading image at index {idx}: {str(e)}")
            raise

def load_model_and_labels(model_path, label_to_idx_path):
    """Load the trained model and label mapping"""
    try:
        # Load label mapping
        with open(label_to_idx_path, 'r') as f:
            label_to_idx = json.load(f)

        # Create inverse mapping
        idx_to_label = {v: k for k, v in label_to_idx.items()}

        # Initialize model
        model = models.resnet50(weights=None)
        model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(model.fc.in_features, len(label_to_idx))
        )

        # Load trained weights
        model.load_state_dict(torch.load(model_path))

        return model, idx_to_label

    except Exception as e:
        logger.error(f"Error loading model and labels: {str(e)}")
        raise

def predict_images(test_set_path, model_path, label_to_idx_path, batch_size,
                  prediction_col_name, output_path, feature_col='Image'):
    """
    Make predictions on test images and save results

    Parameters:
    - test_set_path: path to test CSV file
    - model_path: path to trained model weights
    - label_to_idx_path: path to label mapping JSON
    - batch_size: batch size for predictions
    - prediction_col_name: name for the new predictions column
    - output_path: path to save predictions CSV
    - feature_col: name of column containing image URLs

    Returns:
    - result_df: DataFrame with predictions
    - execution_time: Time taken for predictions in seconds
    - prediction_cost: Cost of predictions based on execution time
    """
    try:
        # Start timing
        start_time = time.time()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {device}")

        # Load test data
        test_df = pd.read_csv(test_set_path)
        logger.info(f"Loaded test set with {len(test_df)} images")

        # Create transforms for test images
        test_transform = transforms.Compose([
            transforms.Resize((224, 224)),  # Standard ResNet input size
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])
        ])

        # Create dataset and dataloader
        test_dataset = TestImageDataset(test_set_path, feature_col, test_transform)
        test_loader = DataLoader(test_dataset,
                               batch_size=batch_size,
                               shuffle=False,
                               num_workers=4)

        # Load model and label mapping
        model, idx_to_label = load_model_and_labels(model_path, label_to_idx_path)
        model = model.to(device)
        model.eval()

        # Make predictions
        predictions = []
        with torch.no_grad():
            for batch_images, batch_indices in test_loader:
                batch_images = batch_images.to(device)
                outputs = model(batch_images)
                _, predicted = torch.max(outputs.data, 1)

                # Convert indices to labels
                batch_predictions = [idx_to_label[idx.item()]
                                  for idx in predicted]

                # Store predictions with their indices
                for idx, pred in zip(batch_indices, batch_predictions):
                    predictions.append((idx.item(), pred))

        # Sort predictions by index to maintain original order
        predictions.sort(key=lambda x: x[0])
        predicted_labels = [pred[1] for pred in predictions]

        # Add predictions to dataframe
        test_df[prediction_col_name] = predicted_labels

        # Save results
        test_df.to_csv(output_path, index=False)

        # Calculate execution time and cost
        execution_time = time.time() - start_time
        prediction_cost = 0.000281392488 * execution_time

        logger.info(f"Predictions saved to {output_path}")
        logger.info(f"Prediction time: {execution_time:.2f} seconds")
        logger.info(f"Prediction cost: ${prediction_cost:.6f}")

        return test_df, execution_time, prediction_cost

    except Exception as e:
        logger.error(f"Error in prediction pipeline: {str(e)}")
        raise

absolute_path = "/content/gdrive/My Drive/Projects/ResNet/"

if __name__ == "__main__":
    test_params = {
        'test_set_path': absolute_path + 'Datasets/test_set_100.csv',
        'model_path': absolute_path + 'model_lr_0.0001_bs_32_es_100.pth',
        'label_to_idx_path': absolute_path + 'model_lr_0.0001_bs_32_es_100_label_to_idx.json',
        'batch_size': 32,
        'prediction_col_name': 'ResNet50-Predictions',
        'output_path': absolute_path + 'Datasets/test_set_100_with_predictions.csv'
    }

    # Run predictions
    result_df, execution_time, prediction_cost = predict_images(**test_params)

    print("\nPrediction Results Summary:")
    print(f"Total prediction time: {execution_time:.2f} seconds")
    print(f"Total prediction cost: ${prediction_cost:.6f}")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-8-44ce3043eb9f>:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))



Prediction Results Summary:
Total prediction time: 25.74 seconds
Total prediction cost: $0.007242
